In [1]:
import json
with open('songs.json', 'r') as json_file:
    data = json.load(json_file)

In [6]:
data[0]

{'songName': 'Hurt',
 'spotifyLink': 'https://open.spotify.com/track/1lo9k4PrxFd5Np4cAxXoKo?si=881892db7eaa4a71',
 'genre': 'Rock',
 'lyrics': "I hurt myself today\nTo see if I still feel\nI focus on the pain\nThe only thing that's real\nThe needle tears a hole\nThe old familiar sting\nTry to kill it all away\nBut I remember everything\nWhat have I become?\nMy sweetest friend\nEveryone I know goes away\nIn the end\nAnd you could have it all\nMy empire of dirt\nI will let you down\nI will make you hurt\nI wear this crown of thorns\nUpon my liar's chair\nFull of broken thoughts\nI cannot repair\nBeneath the stains of time\nThe feelings disappear\nYou are someone else\nI'm still right here\nWhat have I become?\nMy sweetest friend\nEveryone I know goes away\nIn the end\nAnd you could have it all\nMy empire of dirt\nI will let you down\nI will make you hurt\nIf I could start again\nA million miles away\nI would keep myself\nI would find a way",
 'mood': '100stress'}

In [7]:
# convert prediction to value

def pred_to_mh(pred):
    label_mapping = {
        0: "Stressed",
        1: "Depressed",
        2: "Neutral",
        3: "Happy",
        4: "Studious"
    }

    return label_mapping[pred]

result = pred_to_mh(1)

In [8]:
result

'Depressed'

In [ ]:
word_list = {
    "Stressed": 
        ["stress", "uneasy", "nervous", "discomfort", "inconvenient", "Anxious", "Exhausting", "Intense", 
         "Panic", "Unmanageable", "Breakdown"],
    
    "Depressed":
        ["Melancholic", "Somber", "Blue", "Gloomy, depression", "rain", "Overwhelming", "Desperate", 
         "Suicidal", "Paralyzed", "Incapacitating", "Unbearable", "Devastated", "cry", "tears", "pain"],
    
    "Neutral":
        ["bored", "simple", "pleasant", "normal", "neutral"],
        
    "Happy":
        ["Stable", "Calm", "Satisfied", "Happy", "Content", "Manic", "Euphoric", "Ecstatic", "Party",
         "Dance", "excited"],
        
    "Studious":
        ["smart", "study", "brain", "school", "grades", "graduation", "homework", "scholar", "work",
         "writing", "teacher", "students", "reading", "intelligent", "education"]
}